In [ ]:
import os
import sys
sys.path.append("/mnt/scratch/henrypurcell/N3AS/Emu_scripts/spinflip")
import yt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import h5py
from constants import c, M_p, M_3flavor, G, hbar
import spin_flip_tools as sft
import gellmann as gm
import four_current as fc
import matrix
import constants
import diagonalizer as dg
import time_plots as tp
import merger_grid as mg
import glob
import concurrent
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator,LogLocator)
from matplotlib import cm, colors
import matplotlib.axes as ax
from mpl_toolkits.mplot3d import Axes3D
from four_current import four_current, read_gradients
from matrix import visualizer
import argparse
#import latex
from itertools import product

# For ease of development, reload the relevant modules to make sure they are up to date.
import importlib
importlib.reload(constants)
importlib.reload(sft)
importlib.reload(gm)
importlib.reload(fc)
importlib.reload(matrix)
importlib.reload(dg)
importlib.reload(tp)
importlib.reload(mg)

################
# plot options #
################
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=True)

mpl.rcParams['xtick.major.size'] = 7
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.size'] = 4
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['xtick.top'] = True

mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['axes.grid'] = False

np.set_printoptions(precision=16)

#####################
###plot parameters###\
#####################



#location to save png's
plot_output_file = "/mnt/scratch/henrypurcell/N3AS/5-figures_final/"

#resolutions
resolution = 10

merger_data_filename = "/mnt/scratch/shared/2-orthonormal_distributions/model_rl0_orthonormal_rotated.h5"
emu_data_loc = "/mnt/scratch/shared/3-Henry_NSM_box_selected/"
gradient_filename = "/mnt/scratch/shared/4-gradients/gradients_start_i117-134_j64-78_k14-17.h5"
gradient_filename_end = "/mnt/scratch/shared/4-gradients/gradients_end_i117-134_j64-78_k14-17.h5"
#3 z-values of cross sections to plot
zvals = [8,15,99]

#location of specific point to analyze
location=[126,69,15] #will appear in middle plot in contour_plot hence zvals[1]
emu_filename = emu_data_loc + "i{:03d}".format(location[0])+"_j{:03d}".format(location[1])+"_k{:03d}".format(location[2])+"/allData.h5"

#boundaries of regions over which gradients have been computed (for adiabaticity analysis)
#nth item in list corresponds to nth cross section (currently all are the same)
xmins = [75,75,75]
xmaxs = [83,83,83]
ymins = [67,67,67]
ymaxs = [77,77,77]

#neutrino momentum
p_abs = 1e7 # eV



In [ ]:
######################################
### PART 1: 3-FLAVOR, 3D SPIN-FLIP ###
######################################

#compute relevant SpinParams object for the given point
SP_0 = sft.SpinParams(0,
                      emu_filename,
                      merger_data_filename,
                      location,
                      p_abs,
                      resonance_type = 'simplified',
                      gradient_filename = gradient_filename)


In [ ]:
### Angular plot of Eigenvector Resonance value (1-|L-R|) and zoomed in version of the same ###
resolution = 300
SP_0.angularEigenvectorPlot(resolution, resolution,
                            value = 'Omega',
                            phi_optimal = np.pi, theta_optimal=2.33,
                            zoom = 0.025,
                            resonant_band_thetas=[ 2.341995193081677,2.3178148626917529,2.332116965261659],
                            shift = [0,0],
                            vmax = -9.256, vmin = -14.035,
                            zoom_resolution = resolution,
                            initvector = 'default', 
                            method = 'Nelder-Mead',
                            bounds =[(np.pi/2, np.pi)], 
                            linearPlot = False,
                            savefig = plot_output_file + "angular_resonance_init",
                            fs = 20)



In [ ]:
### Linear plot of Eigenvector resonance value vs theta for phi = 3/4 pi (shows 1D phi-slice of zoomed in picture from previous plot)###
# this identifies resonance bands in the channels given in flavor-resonances with the given colors. There are 9 resonance bands total, and we can show 3
# after showing 3 resonance bands we can plot their evolution in the next plot to show they are resonant.
resolution = 350
SP_0.linearEigenvectorPlot( 6*resolution,  
                            initvector = None, value = 'Omega',
                            zoom_on_vector = 2.33,
                            zoom = 0.033, shift = 0, phi_optimal= np.pi,
                            method = 'Nelder-Mead',
                            vmax = 70,
                            legend_loc = (1.01,0.73),
                            bounds =[(np.pi/2, np.pi)],
                            extra_lines = None, extra_init_vectors = None,
                            flavor_resonances = [(0,0,'deepskyblue'), (2,2,'limegreen'), (0,2,'magenta')],
                            savefig = plot_output_file + 'linear_eigenvector_plot_init',
                            fs = 20)

In [ ]:
### Show components of H to connect resonance bands to crossings ###
SP_0.H_component_plotter(0,
                         [0,1,3,4],
                         300,
                         xlims = [[0,np.pi],[2.2,2.5]],
                         savefig = plot_output_file + 'H_components_init',)

In [ ]:
### Compute resonant thetas on resonance bands for the next plot, and find the resonant angular width according to generalized resonance condition ###
find_res_regions = SP_0.findResonantRegions(theta_resolution = 1000, 
                                            phi_optimal = np.pi,
                                            min_dist_between_peaks = 20,
                                            limits = [2.33-0.04, 2.33+0.04],
                                            resonance_threshold = 1,
                                            max_peak_count = 3,
                                            method = 'Nelder-Mead',
                                            makeplot = False,
                                            printvalues = True)
# print total resonant width that will be used in description of the plot
resonant_width = find_res_regions['total_width']
resonant_thetas = find_res_regions['resonant_thetas']
solid_angle = 2*np.pi*resonant_width*np.sin(resonant_thetas[1])
print('total initial resonant width = ', resonant_width)
print('total initial resonant solid angle = ' , solid_angle) 


In [ ]:
### compute resonant Hamiltonians and find Timescales
# first compute the resonant thetas, vectors and Hamiltonians corresponding to these thetas
resonant_states = SP_0.resonant_states(resonant_thetas, P = True)
resonant_Hs = np.array([SP_0.H(float(theta),np.pi) for theta in resonant_thetas])

for H in resonant_Hs: #find oscillation timescale
    D = dg.Diagonalizer(H)
    timescale = D.timescale/2E-6
    print(str(timescale) + ' microseconds')


In [ ]:
### Single-Point Adiabaticity plot: shows adiabaticity and gradient vs azimuthal angle over resonance band for selected point ###
SP_0.azimuthalGradientsPlot(phi_resolution = 1000,  
                            vmax = 30,
                            savefig = plot_output_file + 'azimuthal_gradients_init')



### Adiabaticity width for single-point: prints and returns the angular width of the adiabatic region on the resonance band at the given point
# We'll quote this value when describing the plot
adiabatic_width = SP_0.findAdiabaticRegions(phi_resolution = resolution,
                          min_dist_between_peaks = 1,
                          adiabaticity_threshold = 1,
                          max_peak_count = 3, 
                          method = 'Nelder-Mead')

print('total initial adiabatic width = ', adiabatic_width)
print('total initial resonant and adiabatic solid angle = ' , solid_angle* adiabatic_width/(2*np.pi))



In [ ]:
###############################
### PART 2: FFI SIMULATION  ###
###############################

In [ ]:
### time plot of H_LR increasing during instability in z direction ###
TP = tp.TimePlots([0,150], emu_filename,  merger_data_filename, location, p_abs)

TP.plot( quantity = 'H_LR_components', 
        thetas_and_phis = [(0.005,0),(np.pi/2,0)], 
        labels = (r'$\theta = 0$',r'$\theta = \pi/2$'),
        savefig = plot_output_file + 'hlr_time_evolution',
        vmax = [None, 5E-10])


In [ ]:
### Compute evolved SP object, print time of new plots
#try to load for all values of t until one doesnt return an error
SP_e = sft.SpinParams(250, emu_filename, merger_data_filename, location, p_abs, resonance_type = 'simplified', gradient_filename = gradient_filename_end)
print('time of new plots = ', SP_e.t_seconds*1E9)


In [ ]:
### Evolved angular eigenvector plot showing resonant bands after FFI
SP_e.angularEigenvectorPlot(resolution,resolution,
                            value = 'Omega',
                            phi_optimal = np.pi, theta_optimal=2.4,
                            zoom = 0.3,
                            shift = [0,0],
                            vmax = -9.256, vmin = -14.035,
                            zoom_resolution = resolution,
                            initvector = 'default', 
                            method = 'Nelder-Mead',
                            bounds =[(np.pi/2, np.pi)], 
                            linearPlot = False,
                            flavor_resonances=[],
                            savefig = plot_output_file + "angular_resonance_final.png",
                            fs = 20)

In [ ]:
### Compute resonant thetas on resonance bands for the next plot, and find the resonant angular width according to generalized resonance condition ###
find_res_regions = SP_e.findResonantRegions(theta_resolution = 10000, 
                                            phi_optimal = np.pi,
                                            min_dist_between_peaks = 20,
                                            limits = [2.1, 2.6],
                                            resonance_threshold = 1,
                                            max_peak_count = 9,
                                            method = 'Nelder-Mead',
                                            makeplot = False,
                                            printvalues = True)
# print total resonant width that will be used in description of the plot
resonant_width = find_res_regions['total_width']
resonant_thetas = find_res_regions['resonant_thetas']
solid_angle = 2*np.pi*resonant_width*np.sin(resonant_thetas[1])
print('total initial resonant width = ', resonant_width)
print('total initial resonant solid angle = ' , solid_angle) 


In [ ]:
resolution = 100
### Evolved Linear plot of Eigenvector resonance value vs theta for phi = 3/4 pi (shows 1D phi-slice of zoomed in picture from previous plot)###

SP_e.linearEigenvectorPlot( 20*resolution,  
                            initvector = None, value = 'Omega',
                            zoom_on_vector = 2.4,
                            zoom = 0.32, shift = 0, phi_optimal= np.pi,
                            method = 'Nelder-Mead',
                            vmax = 40,
                            legend_loc = (1.01,0.73),
                            bounds =[(np.pi/2, np.pi)],
                            extra_lines = resonant_thetas, extra_init_vectors = None,
                            flavor_resonances = None,
                            # flavor_resonances = [(0,0,'deepskyblue'), (2,2,'limegreen'), (0,2,'magenta')],
                            savefig = plot_output_file + 'linear_eigenvector_plot_final',
                            fs = 20)

In [ ]:
###########################################
###### PART 3: FULL MERGER ANALYSIS #######
###########################################

In [ ]:
### Resonance and FFI Regions Plot: plots resonance and FFI regions on z-cross sections ###
mg.Merger_Grid([6,15, 98],
               merger_data_loc=merger_data_filename,
               p_abs=p_abs
                ).contour_plot(
                            xmin=20, xmax = 180, ymin=20, ymax=180,
                            #x=[None, location[0], None], 
                            #y=[None, location[1], None], 
                            rect_xmin = [None, location[0]-9,None], rect_xmax = [None, location[0]+8,None],
                            rect_ymin = [None, location[1]-5,None], rect_ymax = [None, location[1]+9,None],
                            savefig = plot_output_file + 'merger_grid_init')


In [ ]:
#compute data for the four angular plots in the resonance multiplot
locations = [[80,73,99],[126,69,15]]
timesteps = [0,250]

emu_filenames = []
emu_filenames.append("/mnt/scratch/shared/3-Henry_NSM_box/" + 
                         "i{:03d}".format(locations[0][0])   \
                         +"_j{:03d}".format(locations[0][1]) \
                         +"_k{:03d}".format(locations[0][2])
                         +"/allData.h5")
                    
emu_filenames.append(emu_data_loc + 
                         "i{:03d}".format(locations[1][0])   \
                         +"_j{:03d}".format(locations[1][1]) \
                         +"_k{:03d}".format(locations[1][2])
                         +"/allData.h5")

SP_list = []
for i in range(2):
    for j in range(2):
        SP_list.append(sft.SpinParams(timesteps[j],
                                  emu_filenames[i],
                                  merger_data_filename,
                                  locations[i],
                                  p_abs,
                                  resonance_type = 'simplified',
                                  gradient_filename = None))

data_list = []
for SP in SP_list:
    data_list.append(SP.angularEigenvectorPlot(resolution, resolution,
                            value = 'Omega',
                            phi_optimal = np.pi, theta_optimal=2.33,
                            returndata = True,
                            vmax = -9.256, vmin = -14.035,
                            zoom_resolution = resolution,
                            initvector = 'default', 
                            fs = 20))



In [ ]:
### resonance band multiplot showing general distribution of bands for 2 selected points
sft.angular_eigenvector_multiplot(data_list, vmax = -9.256, vmin = -14.035,
                                  savefig = plot_output_file + 'angular_resonance_multiplot')

In [ ]:
### Angular Plot of H_LR, before and after simulation ###
sft.multi_HLR_Plotter( 
                    0, 150,
                    emu_filename,
                    merger_data_filename,
                    location,
                    p_abs,
                    40,
                    30, 
                    resonance_type = 'simplified',
                    savefig= plot_output_file + 'angular_hlr_init',
                    use_gm=[1,0],
                    )


In [ ]:
### Adiabaticity Plot: plots average value of adiabaticity on resonance band at each grid cell###
#(averaging over azimuthal anglues, picking phi_resolution of these at each cell)
#Plots for z-cross sections - first input is a list of z's to plot. We can do 3 cross sections for the paper
sft.Gradients(gradient_filename, 
              merger_data_filename
              ).plotAdiabaticities(zvals,
                                   emu_data_loc, 
                                   p_abs,
                                   1E-11, 1E-7,
                                   savefig = plot_output_file + 'adiabaticity_grid_init')
